# Routing optimization notebook
In this notebook we will find the collection of restaurants from each country that are as close as possible to one another.

- First Part:
    1. Getting the distance matrix
    2. Getting the selected restaurants
    3. Optimizing the route for those restaurants
    4. Getting the directions from each restaurant to the next

- Second part (Almost the same as the first but with Techspace as origin point):
    1. Getting the distance matrix
    2. Getting the restaurants for each country closest to Techspace
    3. Getting the directions from Techspace to the restaurants

- Third part - Route optimization by continent

In [2]:
# Importing library from google ortools for the routing optimization part

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_parameters_pb2

# Importing othe libraries that we will need later on

import pandas as pd
import requests
import json
import plotly.express as px
import math
import plotly.graph_objects as go
import getpass
import re



### Writing the tokes securely for the Here API
This API will be used for calculating the way walking or by public transport to get from one place to another

In [3]:
app_id=getpass.getpass()

········


In [4]:
app_code=getpass.getpass()

········


# 1. Selecting one restaurant per country / area

The selected ones should be as close as possible to one another. Current limitation of distance matrix is that distances are calculated as eucledian distances, and are not fully representative of the real distance in cities.

In [96]:
berlin=pd.read_pickle("data/clean_restaurants.pkl")

In [97]:
def compute_euclidean_distance_matrix(locations):
    """Creates callback to return distance between points."""
    distances = {}
    for from_counter, from_node in enumerate(locations):
        distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter:
                distances[from_counter][to_counter] = 0
            else:
                # Euclidean distance
                distances[from_counter][to_counter] = (int(
                    math.hypot((from_node[0] - to_node[0]),
                               (from_node[1] - to_node[1]))))
    return distances

In [98]:
rest=berlin.copy()
rest['new_lat']=rest['lat']*(10**6)
rest['new_lng']=rest['lng']*(10**6)

rest['new_lat']=rest['new_lat'].astype(int)
rest['new_lng']=rest['new_lng'].astype(int)

#Subsetting the df to the restaurants that we are interested in
df1=rest[rest['country']==1]
df2=rest[rest['area']==1]
df3=df1.append(df2,ignore_index=True)

# making a list of all the countries and areas that we have to visit
represented_countries=list(df3[df3["country"]==1]["new_category"].unique())
represented_areas=list(df3[df3["area"]==1]["new_category"].unique())
represented=represented_areas+represented_countries

# setting the starting point the techspace and creating a list of tuples with the lat and lon of each restaurant
techspace=(52503842,13407526)
locations=list(zip(df3['new_lat'],df3['new_lng']))
locations.insert(0,techspace)

# first iteration
list_paises=[]
ruta=pd.DataFrame(columns=df3.columns)

dis=compute_euclidean_distance_matrix(locations)

dis_df=pd.DataFrame(dis)
# We take the location with the minimum distance to our location 0 (Techspace in iteration 0, the previous restaurant for the rest of iterations)
minimum=dis_df[dis_df>0].iloc[0].min()
id_res=dis_df[dis_df[0]==minimum].index

ruta=ruta.append(df3.iloc[id_res-1],ignore_index=True)
pais=df3.iloc[id_res-1]['new_category'].values[0]

list_paises.append(pais)
pais_lat=df3.iloc[id_res-1]['new_lat'].values[0]
pais_lng=df3.iloc[id_res-1]['new_lng'].values[0]

# following iterations
i=0 # variable for control flow

while set(list_paises)!=set(represented):
    # subsetting df by eliminating the country to where the previous restaurant belonged to
    df3=df3[df3['new_category']!=pais]
    previous_rest=(pais_lat,pais_lng)
    locations=list(zip(df3['new_lat'],df3['new_lng']))
    locations.insert(0,previous_rest)
    
    dis=compute_euclidean_distance_matrix(locations)
    dis_df=pd.DataFrame(dis)
    minimum=dis_df[dis_df>0].iloc[0].min()
    id_res=dis_df[dis_df[0]==minimum].index
    
    ruta=ruta.append(df3.iloc[id_res-1],ignore_index=True)
    pais=df3.iloc[id_res-1]['new_category'].values[0]
    
    list_paises.append(pais)
    pais_lat=df3.iloc[id_res-1]['new_lat'].values[0]
    pais_lng=df3.iloc[id_res-1]['new_lng'].values[0]
    
    i+=1
    if i>=100:
        break
    

In [99]:
ruta.head()

,name,lat,lng,query,category,new_category,country,area,continent,new_lat,new_lng
0,Akçaabat Köftesi,52.504879,13.407337,BBQ Joint,Turkish Restaurant,Turkish,1,0,Asia,52504879,13407337
1,Pacifico,52.503847,13.409893,Korea,Korean Restaurant,Korean,1,0,Asia,52503847,13409892
2,King's Chicken,52.503439,13.411943,Iran,Persian Restaurant,Lebanese,1,0,Asia,52503439,13411943
3,Shishi,52.502155,13.411516,Israel,Israeli Restaurant,Israeli,1,0,Asia,52502155,13411516
4,Tourlou,52.500637,13.413470,France,French Restaurant,French,1,0,Europe,52500637,13413470


We proceed to define a dictionary of color for consistency purposes while plotting

In [100]:
coloring={
    "Asia":"dimgray",
    "North America":"red",
    "Oceania":"mediumslateblue",
    "Europe":"orange",
    "South America":"lime",
    "Africa":"deepskyblue",
             }

In [101]:
size=ruta['country'].astype(int).apply(lambda x:x*0.1)
fig=px.scatter_mapbox(ruta,lat='lat',lon='lng',hover_name="name",opacity=1,color="continent",size_max=6,size=size,color_discrete_map=coloring,zoom=10,template='simple_white')
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":1,"t":0,"l":1,"b":0})
fig.for_each_trace(
    lambda trace: trace.update(name=trace.name.replace("continent=", "")),
)
fig.update_layout(
    showlegend=True,
    legend_orientation="h",
    legend_y=0.0,
    legend_x=0.18,
    template="plotly_white"
)

fig.add_trace(go.Scattermapbox(lat=(52.503842,52.503842),lon=(13.4075,13.4075),showlegend=False))
fig.add_trace(go.Scattermapbox(showlegend=False,lat=ruta['lat'],lon=ruta['lng'],mode = "lines",marker=dict(size=12,color='DarkSlateGrey'
                                                                                          )))

fig.show()

## Optimizing the route

We see in this map above that the route is not optimized for the selected restaurants. This problem becomes a Traveling Salesman Problem (TSP). There are multiple algorithms developed for this purpose, here the google algorithm will be used. We have to make small changes to it so we can recover the optimal route. Now instead of printing, the print_solution function will return a long string.
https://developers.google.com/optimization/routing/tsp

In [102]:
# setting the origin to be used in the algorithm
techspace=(52503842,13407526)
locations=list(zip(ruta['new_lat'],ruta['new_lng']))
locations.insert(0,techspace)

In [103]:
def create_data_model(locations):
    matrix={}
    matrix['locations']=locations
    matrix['num_vehicles'] = 1
    matrix['depot'] = 0
    return matrix

def print_solution(manager, routing, assignment):
    """Prints assignment on console."""
    #print('Objective: {}'.format(assignment.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    #print(plan_output)
    plan_output += 'Objective: {}m\n'.format(route_distance)
    return plan_output

def main(locations):
    """Entry point of the program."""
    # Instantiate the data problem.
    matrix = create_data_model(locations)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(matrix['locations']),
                                           matrix['num_vehicles'], matrix['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    distance_matrix = compute_euclidean_distance_matrix(matrix['locations'])

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if assignment:
        sol=print_solution(manager, routing, assignment)
        return sol


if __name__ == '__main__':
    sol=main(locations)

Cleaning the solution of optimizing the route, so we can continue using it

In [104]:
def cleaning_sol(sol,ruta,locations):
    # extracting the indexes from the log string that the function returns
    guide=sol.split(' -> ')
    guide.pop(0)
    guide.insert(0,'0')
    guide=guide[0:len(guide)-1]
    # converting the locations tuples into a df astype int and sorting the locations by the order of the indexes that the function returned
    df_loc=pd.DataFrame(locations)
    guide2=list(map(int,guide))
    sorted_guide=pd.DataFrame(df_loc,index=guide2)
    # merging the df with the original one so we can have the name of the restaurants for each location
    sorted_guide.rename(columns={0: "new_lat", 1: "new_lng"},inplace=True)
    final_guide=sorted_guide.merge(ruta,how='left',on=['new_lat','new_lng'])
    final_guide.head()
    # adding our origin point to the df
    final_guide.fillna('Techspace',inplace=True)
    final_guide.loc[0,'lat']=52503842/(10**6)
    final_guide.loc[0,'lng']=13407526/(10**6)
    final_guide.loc[0,'country']=1
    
    last=final_guide.iloc[0].to_frame().transpose()
    final_guide=final_guide.append(last,ignore_index=True)
    return final_guide

In [105]:
final_guide=cleaning_sol(sol,ruta,locations)

Plotting the optimized route

In [106]:
def plot_optimized(final_guide):
    size=final_guide['country'].astype(int).apply(lambda x:x*0.1)
    fig=px.scatter_mapbox(final_guide,lat='lat',lon='lng',hover_name='name',opacity=1,color="continent",size_max=6,size=size,color_discrete_map=coloring,zoom=10,template='simple_white')
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":1,"t":0,"l":1,"b":0})
    fig.for_each_trace(
        lambda trace: trace.update(name=trace.name.replace("continent=", "")),
    )
    fig.update_layout(
        showlegend=True,
        legend_orientation="h",
        legend_y=0.0,
        legend_x=0.18,
        template="plotly_white"
    )
    
    fig.add_trace(go.Scattermapbox(showlegend=False,lat=final_guide['lat'],lon=final_guide['lng'],mode = "lines",marker=dict(size=12,color='DarkSlateGrey'
                                                                                              )))
    
    fig.show()

In [50]:
plot_optimized(final_guide)

### Calculating the time needed for "traveling around the globe" without leaving berlin

Storing the locations of the restaurants as tuple in the optimized order

In [16]:
final_restaurants=list(zip(final_guide['lat'],final_guide['lng']))

We call the API of Here to get the directions by foot or by public transport to get from one place to another

In [17]:
# Defining the df where we will store the info
guide_directions=pd.DataFrame(columns=['Origin','Destination','Time (min)','Summary','Directions'])
# we call the api for each par of locations (origin, destination)
for x in range(len(final_restaurants)-1):
    origin=final_guide.iloc[x]['name']
    destination=final_guide.iloc[x+1]['name']
    start=str(final_restaurants[x][0])+','+str(final_restaurants[x][1])
    end=str(final_restaurants[x+1][0])+','+str(final_restaurants[x+1][1])
    
    #calling API
    url=f"https://route.api.here.com/routing/7.2/calculateroute.json?app_id={app_id}&app_code={app_code}&departure=2019-12-07T12:00:00&waypoint0=geo!{start}&waypoint1=geo!{end}&mode=shortest;publicTransport;traffic:disabled"
    response=requests.get(url)
    data=response.json()
    # extracting the directions from the response
    directions=''
    for x in range(len(data['response']['route'][0]['leg'][0]['maneuver'])):
        ins=data['response']['route'][0]['leg'][0]['maneuver'][x]['instruction']
        directions=directions+ins
    # using regex to clean the response that is in html format
    directions=re.split('(?<=\<).*?(?=\>)',directions)
    directions=" ".join(directions)
    directions="".join(directions.split('< >'))
    #extracting the public transport lines if any needed
    public_lines=''
    try:
        for x in range(len(data['response']['route'][0]['publicTransportLine'])):
            line=data['response']['route'][0]['publicTransportLine'][x]['lineName']
            if x==(len(data['response']['route'][0]['publicTransportLine'])-1):
                public_lines=public_lines+line
            else:
                public_lines=public_lines+line+', '
    except:
        continue
        
    # adding the extracted and transformed info into the dataframe
    time=data['response']['route'][0]['summary']['baseTime']/60
    info=pd.Series((origin,destination,time,public_lines,directions),index=list(guide_directions.columns)).to_frame().transpose()
    new_df=pd.DataFrame(data=info,columns=list(guide_directions.columns))
    guide_directions=guide_directions.append(new_df,ignore_index=True)

#### Saving the dataframe with the directions as pickle so we dont need to call the API again

In [19]:
guide_directions.to_pickle('data/directions.pkl')

In [20]:
guide_directions_read=pd.read_pickle('data/directions.pkl')
guide_directions_read.head()

,Origin,Destination,Time (min),Summary,Directions
0,Techspace,Finnish Pulled Pork,24.6167,"U8, U7",Head northeast on Lobeckstraße. Go for 72 m.Tu...
1,Finnish Pulled Pork,Munch's Hus,21.1333,U7,Head north on Zossener Straße. Go for 74 m.Tur...
2,Munch's Hus,Panama Restaurant & Bar,14.3333,,Head north on Bülowstraße. Go for 24 m.Take th...
3,Panama Restaurant & Bar,Saeson Nordic Cuisine,7.7,,Head west on Pohlstraße. Go for 114 m.Turn rig...
4,Saeson Nordic Cuisine,Ayan Filipino Streetfood,0.516667,,Head southwest on Potsdamer Straße. Go for 31 ...


# Hours needed to travel around the world!

Assuming that you dont make any stop in the restaurants ...

In [21]:
guide_directions['Time (min)'].sum()/60

17.419722222222223

In [22]:
directions # Example of the intructions that one would be able to see

'Head northeast on Oranienstraße. Go for 71 m.Turn right onto Oranienstraße. Go for 97 m.Arrive at Lobeckstraße. Your destination is on the left.'

# 2. Looking for the closest restaurant of each country to Techspace

In [92]:
ber=berlin.copy()
ber['new_lat']=ber['lat']*(10**6)
ber['new_lng']=ber['lng']*(10**6)

ber['new_lat']=ber['new_lat'].astype(int)
ber['new_lng']=ber['new_lng'].astype(int)
#Subsetting the df to the restaurants that we are interested in
df1_ber=ber[ber['country']==1]
df2_ber=ber[ber['area']==1]
df3_ber=df1_ber.append(df2_ber,ignore_index=True)

# making a list of all the countries and areas that we have to visit
represented_countries=list(df3_ber[df3_ber["country"]==1]["new_category"].unique())
represented_areas=list(df3_ber[df3_ber["area"]==1]["new_category"].unique())
represented_ber=represented_areas+represented_countries

# setting the starting point the techspace and creating a list of tuples with the lat and lon of each restaurant
techspace=(52503842,13407526)
locations=list(zip(df3_ber['new_lat'],df3_ber['new_lng']))
locations.insert(0,techspace)

# first iteration
list_paises=[]
ruta_ber=pd.DataFrame(columns=df3_ber.columns)

dis=compute_euclidean_distance_matrix(locations)

dis_df=pd.DataFrame(dis)

minimum=dis_df[dis_df>0].iloc[0].min()
id_res=dis_df[dis_df[0]==minimum].index

ruta_ber=ruta_ber.append(df3_ber.iloc[id_res-1],ignore_index=True)
pais=df3_ber.iloc[id_res-1]['new_category'].values[0]

list_paises.append(pais)

i=0
while set(list_paises)!=set(represented_ber):

    # following iterations
    
    df3_ber=df3_ber[df3_ber['new_category']!=pais]
    
    locations=list(zip(df3_ber['new_lat'],df3_ber['new_lng']))
    locations.insert(0,techspace)
    
    dis=compute_euclidean_distance_matrix(locations)
    dis_df=pd.DataFrame(dis)

    minimum=dis_df[dis_df>0].iloc[0].min()
    id_res=dis_df[dis_df[0]==minimum].index
    
    ruta_ber=ruta_ber.append(df3_ber.iloc[id_res-1],ignore_index=True)
    pais=df3_ber.iloc[id_res-1]['new_category'].values[0]
    
    list_paises.append(pais)
    
    i+=1
    if i>=100:
        break

In [93]:
ruta_ber.head()

,name,lat,lng,query,category,new_category,country,area,continent,new_lat,new_lng
0,Akçaabat Köftesi,52.504879,13.407337,BBQ Joint,Turkish Restaurant,Turkish,1,0,Asia,52504879,13407337
1,Pacifico,52.503847,13.409893,Korea,Korean Restaurant,Korean,1,0,Asia,52503847,13409892
2,Shishi,52.502155,13.411516,Israel,Israeli Restaurant,Israeli,1,0,Asia,52502155,13411516
3,King's Chicken,52.503439,13.411943,Iran,Persian Restaurant,Lebanese,1,0,Asia,52503439,13411943
4,Tapas y más,52.510118,13.406298,Spain,Tapas Restaurant,Spanish,1,0,Europe,52510117,13406298


Plotting the restaurants closest to Techspace

In [94]:
size=ruta_ber['country'].astype(int).apply(lambda x:x*0.1)
fig=px.scatter_mapbox(ruta_ber,lat='lat',lon='lng',hover_name="name",opacity=1,color="continent",size_max=7,size=size,color_discrete_map=coloring,zoom=10,template='simple_white')
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":1,"t":0,"l":1,"b":0})
fig.for_each_trace(
    lambda trace: trace.update(name=trace.name.replace("continent=", "")),
)
fig.update_layout(
    showlegend=True,
    legend_orientation="h",
    legend_y=0.0,
    legend_x=0.18,
    template="plotly_white"
)

fig.add_trace(go.Scattermapbox(lat=(52.503842,52.503842),lon=(13.4075,13.4075),showlegend=False))



for x in range(len(ruta_ber)):
    
    ruta_ber_lat=52.503842,ruta_ber.iloc[x]['lat']
    ruta_ber_lng=13.407526,ruta_ber.iloc[x]['lng']
    
    fig.add_trace(go.Scattermapbox(showlegend=False,lat=ruta_ber_lat,lon=ruta_ber_lng,mode = "lines",marker=dict(size=12,color='Grey'),opacity=0.5))

fig.show()

## Calculating the time needed to go to each restaurant from Techspace

In [26]:
final_restaurants_techsp=list(zip(ruta_ber['lat'],ruta_ber['lng']))

In [27]:
guide_directions_from_techsp=pd.DataFrame(columns=['Origin','Destination','Time (min)','Summary','Directions'])
for x in range(len(final_restaurants_techsp)):
    origin='Techspace'
    destination=ruta_ber.iloc[x]['name']
    start='52.503842,13.407526'
    end=str(final_restaurants_techsp[x][0])+','+str(final_restaurants_techsp[x][1])
    #calling API
    url=f"https://route.api.here.com/routing/7.2/calculateroute.json?app_id={app_id}&app_code={app_code}&departure=2019-12-07T12:00:00&waypoint0=geo!{start}&waypoint1=geo!{end}&mode=shortest;publicTransport;traffic:disabled"
    response=requests.get(url)
    data=response.json()
    
    directions=''
    for x in range(len(data['response']['route'][0]['leg'][0]['maneuver'])):
        ins=data['response']['route'][0]['leg'][0]['maneuver'][x]['instruction']
        directions=directions+ins
    
    directions=re.split('(?<=\<).*?(?=\>)',directions)
    directions=" ".join(directions)
    directions="".join(directions.split('< >'))
    
    public_lines=''
    try:
        for x in range(len(data['response']['route'][0]['publicTransportLine'])):
            line=data['response']['route'][0]['publicTransportLine'][x]['lineName']
            if x==(len(data['response']['route'][0]['publicTransportLine'])-1):
                public_lines=public_lines+line
            else:
                public_lines=public_lines+line+', '
    except:
        public_lines='No need of public transport'
        
        
    time=data['response']['route'][0]['summary']['baseTime']/60
    info=pd.Series((origin,destination,time,public_lines,directions),index=list(guide_directions_from_techsp.columns)).to_frame().transpose()
    new_df=pd.DataFrame(data=info,columns=list(guide_directions_from_techsp.columns))
    guide_directions_from_techsp=guide_directions_from_techsp.append(new_df,ignore_index=True)

Exporting the direction as pickle so we don't need to call the API again

In [29]:
guide_directions_from_techsp.to_pickle('data/directions_from_techspace.pkl')

In [30]:
guide_directions_from_techsp_read= pd.read_pickle('data/directions_from_techspace.pkl')
guide_directions_from_techsp_read.head()

,Origin,Destination,Time (min),Summary,Directions
0,Techspace,Akçaabat Köftesi,2.83333,,Head northeast on Lobeckstraße. Go for 72 m.Co...
1,Techspace,Pacifico,3.8,,Head northeast on Lobeckstraße. Go for 72 m.Tu...
2,Techspace,Shishi,8.5,,Head northeast on Lobeckstraße. Go for 72 m.Tu...
3,Techspace,King's Chicken,7,,Head northeast on Lobeckstraße. Go for 72 m.Tu...
4,Techspace,Tapas y más,14.9833,,Head northeast on Lobeckstraße. Go for 72 m.Co...


# 3. Optimizing route for restaurants by continent

In [68]:
continent=berlin.copy()
# converting the coordinates in order not to lose precision when using the optimization algorithm
continent['new_lat']=continent['lat']*(10**6)
continent['new_lng']=continent['lng']*(10**6)

continent['new_lat']=continent['new_lat'].astype(int)
continent['new_lng']=continent['new_lng'].astype(int)

In [91]:
def by_continent(continent_name):

    cont_df=continent[continent['continent']==continent_name]
    #display(cont_df)
    techspace=(52503842,13407526)
    locations=list(zip(cont_df['new_lat'],cont_df['new_lng']))
    locations.insert(0,techspace)
    #print(locations)
    if __name__ == '__main__':
        sol=main(locations)
    #print(sol)
    cont_guide=cleaning_sol(sol,continent,locations)
    #display(cont_guide)
    plot_optimized(cont_guide)

### Taking South America as an example

In [89]:
by_continent('South America')

closest spanish restaurants to me